In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from owlready2 import *

SQLITE3FILE = os.getcwd() + "/4WD_car_db.sqlite3"
IRI = "http://www.semanticweb.org/fenrir/ontologies/2023/5/4WD_car_specific_ontology"

def get_onto():
    default_world.set_backend(filename=SQLITE3FILE, exclusive=False)
    return default_world.get_ontology(IRI).load()

def close():
    default_world.close()

onto = get_onto()

In [ ]:
battery_df = pd.DataFrame(columns=["timestamp","voltage"])

for car in onto.search(is_a = onto.car):
    for battery in car.hasPart:
        if not isinstance(battery, onto.battery): continue
        for measure in battery.hasMeasure:
            battery_df.loc[len(battery_df)] = pd.Series(dtype=object)
            for prop in measure.get_properties():
                for value in prop[measure]:
                    battery_df.loc[len(battery_df)-1, prop.python_name] = value

battery_df = battery_df.drop('charge', axis = 1)
battery_df['diff_voltage'] = battery_df['voltage'].diff()
threshold = 2
battery_df['cycle'] = (battery_df['diff_voltage'] > threshold).cumsum()
battery_df['timestamp'] = battery_df['timestamp'] - battery_df.groupby('cycle')['timestamp'].transform('min')
discharge_cycles = battery_df.groupby('cycle')

In [ ]:
discharge_cycles.head(10)

In [ ]:
fig, ax = plt.subplots()

for n, cycle in discharge_cycles:
    ax.plot(cycle.timestamp, cycle.voltage)

ax.legend()
ax.set_xlabel('Timestamp')
ax.set_ylabel('Voltage')
plt.title('Voltage over Time')

plt.show()

In [ ]:
df = pd.DataFrame(columns=["timestamp"])

for assembly in onto.search(is_a = onto.Assembly):
    for part in assembly.hasPart:
        for measure in part.hasMeasure:
            df.loc[len(df)] = pd.Series(dtype=object)
            for prop in measure.get_properties():
                for value in prop[measure]:
                    df.loc[len(df)-1, prop.python_name] = value

df.drop('position', axis=1)
df.groupby('timestamp').sum().head()

In [ ]:
close()